Grph

Given: A collection of DNA strings in FASTA format having total length at most 10 kbp.
Return: The adjacency list corresponding to O3. You may return edges in any order.

In [ ]:
from Bio import SeqIO
def read_fasta(filename):
    identifiers=[]
    sequences=[]
    for record in SeqIO.parse(filename,"fasta"):#parse fasta using biopython function(record refers to header and seq)
        identifiers.append(record.id)
        sequences.append(str(record.seq))#add sequences of dna to empty list
    return identifiers, sequences#return list of sequences

# check if ending part of first string of length k matches starting part of second string of length k
def overlap_graph(identifiers,sequences, k):
    adjency_list=[]
    for i in range(len(sequences)):
        for j in range(len(sequences)):
            if i!=j:
                if sequences[i][-k:]==sequences[j][:k]:
                    adjency_list.append((identifiers[i],identifiers[j]))#if suffix of i of length  k is equal to prefix of j of length k, a match is found and the edge(the identifier pair)is added to the list
    return adjency_list
def main():
    filename="grph.fasta"
    identifiers, sequences=read_fasta(filename)
    adjency_list=overlap_graph(identifiers, sequences, k=3)
    for edge in adjency_list:
        print(edge[0], edge[1])
main()


Long

Given: At most 50 DNA strings of approximately equal length, not exceeding 1 kbp, in FASTA format (which represent reads deriving from the same strand of a single linear chromosome).
The dataset is guaranteed to satisfy the following condition: there exists a unique way to reconstruct the entire chromosome from these reads by gluing together pairs of reads that overlap by more than half their length.
Return: A shortest superstring containing all the given strings (thus corresponding to a reconstructed chromosome).

In [ ]:
from Bio import SeqIO
def read_fasta(filename):
    sequences=[]
    for record in SeqIO.parse(filename,"fasta"):
        sequences.append(str(record.seq))
    return sequences

#find the maximum overlap between 2 strings
def max_overlap(s1,s2):
    max_o=0
    merged_strings=s1+s2
    for i in range(1,len(s1)):
        if s2.startswith(s1[-i:]):
            if i>max_o:
                max_o=i
                merged_strings=s1+s2[i:]
    return max_o, merged_strings

#function for shortest superstring 
def max_superstring(sequences):
    while len(sequences)>1:
        max_length=-1
        best_pair=(0,0)
        best_merged=""
        for i in range(len(sequences)):
            for j in range(len(sequences)):
                if i!=j:
                    overlap_length, merged=max_overlap(sequences[i], sequences[j])
                    if overlap_length>max_length:
                        max_length=overlap_length
                        best_pair=(i,j)
                        best_merged=merged
    #merge best pair found
        i,j=best_pair
        sequences[i]=best_merged
        sequences.pop(j)
    return sequences[0]
def main():
    filename="long.fasta"
    sequences=read_fasta(filename)
    superstring=max_superstring(sequences)
    print(superstring)

if __name__ == '__main__':
    main()


Tree

Given: A positive integer n(n≤1000) and an adjacency list corresponding to a graph on n nodes that contains no cycles.
Return: The minimum number of edges that can be added to the graph to produce a tree.

In [ ]:
import networkx as nx
def read_input(filename):
    with open(filename,"r") as file:
        lines=file.readlines()
    n=int(lines[0].strip())#n of nodes
    edges=[]
    for line in lines[1:]:
        u,v=map(int, line.strip().split())
        edges.append((u,v))#add edges as tuples
    return n, edges
def min_edges(n, edges):
    G=nx.Graph()
    G.add_nodes_from(range(1,n+1))#add nodes 1 trough n
    G.add_edges_from(edges)#add edges from list
    n_components=nx.number_connected_components(G)#find number of connected components
    return n_components-1#minimum number of edges to connect graph from a tree
def main():
    filename="tree.txt"
    n, edges=read_input(filename)
    final_edges=min_edges(n, edges)
    print(final_edges)

if __name__ == '__main__':
    main()

Deg

In an undirected graph, the degree d(u) of a vertex u is the number of neighbors u has, or equivalently, 
the number of edges incident upon it.
Given: A simple graph with n≤103 vertices in the edge list format.
Return: An array D[1..n] where D[i] is the degree of vertex i.

In [ ]:
#degree of a vertex is the number of edges connected to it(edges are connections between nodes)
def calculate_degrees(filename):
    with open(filename,'r') as file:
        first_line=file.readline().strip()
        n,m=map(int, first_line.split())# each line is a string that we divide into 2 integers
        D=[0]*(n+1)# initialize array of degrees
        for line in file:
            u,v=map(int, line.strip().split())
            D[u]+=1
            D[v]+=1 #edge count increases by 1 each time vertex u or v is connected to a new edge
        return D[1:]#exclude the zero index
filename="deg.txt"
degrees=calculate_degrees(filename)
print(' '.join(map(str, degrees)))

Ddeg

Given: A simple graph with n≤103 vertices in the edge list format.
Return: An array D[1..n]where D[i]is the sum of the degrees of i's neighbors.

In [ ]:
#we calculate first the degree(the number of edges)for each vertex, then calculate for each vertex the sum of degrees of its neighbors
def sum_degrees(filename):
    with open(filename,'r') as file:
        first_line=file.readline().strip()
        n,m=map(int, first_line.split())
        degree=[0]*(n+1)#to initialize array of degrees to keep track of each vertex
        neighbors_list=[[] for _ in range(n + 1)]#list to store neighbors for each vertex
        for line in file:
            u,v=map(int, line.strip().split())
            degree[u]+=1
            degree[v]+=1#as before, for each edge encountered we increment the number of degree for both vertices 
            neighbors_list[u].append(v)
            neighbors_list[v].append(u) #then add vertex v to the list of neighbors of u and viceversa
        D=[0]*(n+1)#result array
        for i in range(1,n+1):
            D[i]=sum(degree[neighbor]for neighbor in neighbors_list[i])#for each vertex i sum the degrees of its neighbors
        return D[1:]
filename="ddeg.txt"
final_result=sum_degrees(filename)
print(' '.join(map(str, final_result)))

Cc

Given: A simple graph with n≤103 vertices in the edge list format.
Return: The number of connected components in the graph.


In [ ]:
#edge list format:The first line contains two numbers, the number of vertices n and the number of edges m, each of the following m
#lines contains an edge given by two vertices.
import networkx as nx
def counter_components(filename):
    with open(filename,'r') as file:
        first_line=file.readline().strip()
        n,_=map(int, first_line.split()) #extract only vertices n, the number of edges m is not needed
        edges=[]#empty list to store edges
        for line in file:
            u,v=map(int, line.strip().split()) 
            edges.append((u,v))
    graph=nx.Graph()
    graph.add_edges_from(edges)#add edges stored in list to the graph
    graph.add_nodes_from(range(1,n+1))#adds allvertices included in the graph
    connected_components=nx.number_connected_components(graph)#function to count connected components
    return connected_components
filename="cc.txt"
tot_connected_components=counter_components(filename)
print(tot_connected_components)
  

Bfs

Given: A simple directed graph with n≤103 vertices in the edge list format.
Return: An array D[1..n] where D[i]is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0).
If i is not reachable from 1set D[i] to −1.

In [ ]:
import networkx as nx
def find_shortest_path(filename):
    with open(filename,'r') as file:
        first_line=file.readline().strip()
        n,_=map(int, first_line.split())
        edges=[]
        for line in file:
            u,v=map(int, line.strip().split()) 
            edges.append((u,v))
    graph=nx.DiGraph()#directed graph
    graph.add_edges_from(edges)
    if 1 in graph.nodes:#compute shortest paths staring from node 1
        shortest_paths=nx.single_source_shortest_path_length(graph, source=1)
    else:
        shortest_paths={}
    distances=[-1]*n#initialize all distances as -1 
    for node, distance in shortest_paths.items():
        distances[node-1]=distance#update distance for reachable nodes, using nodes-1 because graph's nodes
        #are 1-indexed but we want them to be 0-indexed
    return distances
filename="bfs.txt"
result=find_shortest_path(filename)
print(' '.join(map(str, result)))


Dag

Given: A positive integer k≤20and ksimple directed graphs in the edge list format with at most 103 vertices and 3⋅103edges each.
Return: For each graph, output "1" if the graph is acyclic and "-1" otherwise.

In [ ]:

import networkx as nx
def read_input(filename):
    with open(filename,'r') as file:#file contains multiple graphs
        k=int(file.readline().strip())#n of graphs
        graphs=[]
        for _ in range(k):
            line = file.readline().strip()
            while not line:  # Skip empty lines
                line = file.readline().strip()
            n, m=map(int, line.split())#n of nodes and edges
            edges=[]
            for _ in range(m):
                u,v=map(int, file.readline().strip().split()) 
                edges.append((u,v))
            graphs.append(edges)
    return graphs

def check_ciclycity(graphs):
    results=[]
    for edges in graphs:
        graph=nx.DiGraph()
        graph.add_edges_from(edges)
        if nx.is_directed_acyclic_graph(graph):
            results.append(1)
        else:
            results.append(-1)
    return results

def main():
    filename="dag.txt"
    graphs=read_input(filename)
    results=check_ciclycity(graphs)
    print(' '.join(map(str, results)))

if __name__=="__main__":
    main()


Bip

Given: A positive integer k≤20 and ksimple graphs in the edge list format with at most 103 vertices each.
Return: For each graph, output "1" if it is bipartite and "-1" otherwise.


In [ ]:
import networkx as nx

def read_input(filename):
    with open(filename,'r') as file:#file contains multiple graphs
        k=int(file.readline().strip())#n of graphs
        graphs=[]
        for _ in range(k):
            line = file.readline().strip()
            while not line:  # Skip empty lines
                line = file.readline().strip()
                n, m=map(int, line.split())#n of nodes and edges
                edges=[]
                for _ in range(m):
                    u,v=map(int, file.readline().strip().split()) 
                    edges.append((u,v))
                graphs.append(edges)
    return graphs
def check_bipartite(graphs):
    results=[]
    for edges in graphs:
        graph=nx.Graph()
        graph.add_edges_from(edges)
        if nx.is_bipartite(graph):
            results.append(1)
        else:
            results.append(-1)
    return results

def main():
    filename="bip.txt"
    graphs=read_input(filename)
    results=check_bipartite(graphs)
    print(' '.join(map(str, results)))
    
if __name__=="__main__":
    main()


Dij

Given: A simple directed graph with positive edge weights from 1 to 103 and n≤103 vertices in the edge 
list format.
Return: An array D[1..n]where D[i]is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). 
If i is not reachable from 1 set D[i]to −1

In [ ]:
import networkx as nx
def read_input(filename):
    with open(filename,'r') as file:
        first_line=file.readline().strip()
        while not first_line:
            first_line = file.readline().strip()
        n,m=map(int, first_line.split())#extract only nodes, ignoring edges, here not needed
        edges=[]
        for line in file:
            line = line.strip()
            if not line:  # if empty line loop continues to the next
                continue
            u,v,w=map(int, line.strip().split())#u=starting vertex of edge, v=ending vertex of edge, w=weight of edge between u and w vertices 
            edges.append((u,v,w))
    return n, edges
def  dijkstra_shortest_path(filename):
    n, edges=read_input(filename)
    graph=nx.DiGraph()#directed graph
    graph.add_weighted_edges_from(edges)
    if 1 in graph.nodes:#compute shortest paths staring from node 1
        shortest_paths=nx.single_source_dijkstra_path_length(graph, source=1)
    else:
        shortest_paths={}
    D=[-1]*(n+1)#initialize all distancs as -1 
    for node in range(1, n+1):
        if node ==1:
            D[node]=0#if the current vertex is the source vertex, its distance to itslef is equal zero
        elif node in shortest_paths:
            D[node]=shortest_paths[node]#the vertex is reachable from vertex 1 because is found in shortest_paths so we set distance 
            #from this vertex to the value found in shortest_paths
    return D[1:]
def main():
    filename="dij.txt"
    result=dijkstra_shortest_path(filename)
    print(' '.join(map(str, result)))

if __name__ == "__main__":
    main()


Bf

Given: A simple directed graph with integer edge weights from −103 to 103 and n≤103 vertices in 
the edge list format.
Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i
(D[1]=0). If i is not reachable from 1 set D[i]to x.

In [ ]:
#we compute the shortest path from vertex 1 to all the other vertices and build an array with the shortest sequences
import networkx as nx

def read_input(filename):
    with open(filename,'r') as file:
        first_line = file.readline().strip()
        n, m=map(int, first_line.split())# extract n, so vertices and m, so edges
        edges=[]
        for line in file:
            line=line.strip()
            if not line:#skipping empty lines
                continue
            u,v,w=map(int, line.split())
            edges.append((u,v,w))
    return n, edges

def bf_shortest_path(filename):
    n, edges=read_input(filename)
    graph=nx.DiGraph()
    graph.add_weighted_edges_from(edges)
    if 1 in graph.nodes:#compute shortest paths staring from node 1
        shortest_paths=nx.single_source_bellman_ford_path_length(graph, source=1)
    else:
        shortest_paths={}
    D=['x']*(n+1)#initialize distance array with special value x, indicating the unreachable vertices
    for node in range(1, n+1):
        if node ==1:
            D[node]=0
        elif node in shortest_paths:
            D[node]=shortest_paths[node]
    return D[1:]
def main():
    filename="bf.txt"
    result=bf_shortest_path(filename)
    print(' '.join(map(str, result)))
if __name__=="__main__":
    main()


Cte

Given: A positive integer k≤20 and k simple directed graphs with positive integer edge weights and at most 103 vertices 
in the edge list format.
Return: For each graph, output the length of a shortest cycle going through the first specified edge if there is a cycle 
and "-1" otherwise.

In [ ]:
import networkx as nx
def read_input(filename):

    with open(filename,'r') as file:
        first_line=file.readline().strip()
        while not first_line:
            first_line = file.readline().strip()
        k=int(first_line)#extract number of graphs

        graphs=[]
        for _ in range(k):
            line = file.readline().strip()
            while not line:  # Skip empty lines
                line = file.readline().strip()
            n,m=map(int, line.split())#extract vertices and edges out of each graph 

            edges=[]
            for _ in range(m):
                edge_line = file.readline().strip()
                while not edge_line:  # Skip empty lines
                    edge_line=file.readline().strip()
                u, v, w=map(int, edge_line.split())
                edges.append((u,v,w))
            graphs.append(edges)
    return graphs

def shortest_cycle(graphs):
    results=[]
    for edges in graphs:
        u,v,w=edges[0]

        graph=nx.DiGraph()
        graph.add_weighted_edges_from(edges)

        graph.remove_edge(u,v)#the first edge is removed temporarily, since we are finding the shortest cycle, so the for loop goes 
        #into a closed path that starts from the first node, travels through the  other nodes and comes back to the first
        #find shortest paths from v
        shortest_paths=nx.single_source_dijkstra_path_length(graph, source=v, weight='weight')#parameters:graph where you calculate 
        #the shortest path length, source is the starting node from where you count, weigth value referring to edge's value
        if u in shortest_paths: #check if there's a path from v back to u
            cycle_length=shortest_paths[u]+w#if there's a path from v bacj to u, calculate its length
            results.append(cycle_length)
        else:#when no path exists
            results.append(-1)
        graph.add_edge(u,v,weight=w)
    return results

def main():
    filename="cte.txt"
    graphs=read_input(filename)
    results=shortest_cycle(graphs) 
    print(' '.join(map(str, results))) 
if __name__=="__main__":
    main()


Nwc

The task is to use Bellman-Ford algorithm to check whether a given graph contains a cycle of negative weight.
Given: A positive integer k≤20 and k simple directed graphs with integer edge weights from −103 to 103 and n≤103 vertices 
in the edge list format.
Return: For each graph, output "1" if it contains a negative weight cycle and "-1" otherwise.


In [ ]:
import networkx as nx
def read_input(filename):

    with open(filename,'r') as file:
        first_line=file.readline().strip()
        while not first_line:
            first_line = file.readline().strip()
        k=int(first_line)# number of graphs

        graphs=[]
        for _ in range(k):#loop trough graphs
            line = file.readline().strip()
            while not line:  
                line = file.readline().strip()
            n,m=map(int, line.split())#number of vertices and edges out of each graph 

            edges=[]
            for _ in range(m):#loop through edges
                edge_line = file.readline().strip()
                while not edge_line:  
                    edge_line=file.readline().strip()
                u, v, w=map(int, edge_line.split())
                edges.append((u,v,w))
            graphs.append(edges)
    return graphs

def check_negative_weigth_cycle(graphs):#if a negative cycle in the graph exists, function returns 1, otherwise -1
    results=[]
    for edges in graphs:
        graph=nx.DiGraph()
        graph.add_weighted_edges_from(edges)

        if nx.negative_edge_cycle(graph, weight="weight") : #check if there's a negative cycle
            results.append(1)
        else:#when no path exists
            results.append(-1)
    return results
def main():
    filename="nwc.txt"
    graphs=read_input(filename)
    results=check_negative_weigth_cycle(graphs)
    print(' '.join(map(str, results)))
    
if __name__=="__main__":
    main()

Sdag

Given: A weighted DAG with integer edge weights from −103 to 103 and n≤105 vertices in the edge list format.
Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). If i
is not reachable from 1 set D[i] to x.

In [ ]:
#There are two subclasses of graphs that automatically exclude the possibility of negative cycles: graphs without negative edges, 
#and graphs without cycles. We will now see how the single-source shortest-path problem can be solved in just linear time on 
#directed acyclic graphs.
#Perform a sequence of updates (recall Bellman-Ford algorithm) that includes every shortest path as a subsequence. The key source 
# of efficiency is that  the vertices appear in increasing linearized order.
#we linearize  the DAG by depth-first search, and then visit the vertices in sorted order, updating the edges out of each. 
import networkx as nx
def read_input(filename):
    with open(filename,'r') as file:
        first_line=file.readline().strip()
        n,m=map(int, first_line.split())
        edges=[]
        for _ in range(m):
            edge_line=file.readline().strip()
            while not edge_line:
                edge_line=file.readline().strip()
            u,v,w=map(int, edge_line.split())
            edges.append((u,v,w))
    return n, edges

def  shortest_paths(n, edges):
    graph=nx.DiGraph()#directed graph
    graph.add_weighted_edges_from(edges)
    topological_order=list(nx.topological_sort(graph))#list stores vertices in their topological order, so when u->v, u comes before v
    
    D={1:0}#initialize distances indicating they are compared to the source

    for u in topological_order:
        if u in D:
            for v, attr in graph[u].items():
                weight=attr['weight']
                if v not in D or D[u]+weight<D[v]:
                    D[v]=D[u]+weight
    result=[D[i] if i in D else 'x' for i in range(1,n+1)]
    return result
    
def main():
    filename="sdag.txt"
    n, edges=read_input(filename)
    result=shortest_paths(n, edges)
    print(' '.join(map(str, result)))

if __name__ == "__main__":
    main()
